In [12]:
# ---  ИМПОРТ БИБЛИОТЕК И ЗАГРУЗКА ДАННЫХ ---

# 1. Импортируем необходимые инструменты
import pandas as pd  # Библиотека для работы с таблицами (DataFrame)
import numpy as np   # Библиотека для математических операций
from sklearn.neighbors import NearestNeighbors # Наша главная модель (KNN) для поиска соседей
from sklearn.preprocessing import StandardScaler # Инструмент для масштабирования (чтобы рейтинг и жанры были в одном диапазоне)

from google.colab import drive # Библиотека для работы с Гугл Диском
import warnings

# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings("ignore", category=UserWarning) # Убираем красные предупреждения

print("✅ Библиотеки загружены.")

# 2. Подключаем Google Диск
# Когда ты запустишь этот код, появится всплывающее окно.
# Нажми "Разрешить" (Connect to Google Drive), чтобы Colab мог читать твои файлы.
drive.mount('/content/drive')

# 3. Указываем путь к файлу
# ВНИМАНИЕ: Если ты создал папку MovieProject, путь будет таким:
# '/content/drive/MyDrive/MovieProject/IMDb_All_Genres_etf_clean1.csv'
# Если файл просто лежит на диске без папки, то:
# '/content/drive/MyDrive/IMDb_All_Genres_etf_clean1.csv'

✅ Библиотеки загружены.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# 2. Загружаем данные
file_path = '/content/drive/MyDrive/проект/IMDb_All_Genres_etf_clean1.csv'

try:
    df = pd.read_csv(file_path)
    print(f"\n✅ Файл успешно открыт с Google Диска!")
    print(f"Размер: {df.shape}")
    print("\nКолонки:", list(df.columns))
    print(df.head(3))

except FileNotFoundError:
    print(f"❌ Ошибка: Файл по пути '{file_path}' не найден.")
    print("Подсказка: Слева в меню нажми на значок 'Папка' -> drive -> MyDrive.")
    print("Найди свой файл, нажми на него правой кнопкой мыши -> 'Скопировать путь' (Copy path).")
    print("Вставь этот путь в переменную file_path выше.")


✅ Файл успешно открыт с Google Диска!
Размер: (5562, 10)

Колонки: ['Movie_Title', 'Year', 'Director', 'Actors', 'Rating', 'Runtime(Mins)', 'Censor', 'Total_Gross', 'main_genre', 'side_genre']
                                     Movie_Title  Year           Director                                             Actors  Rating  Runtime(Mins) Censor   Total_Gross main_genre          side_genre
0                                        Kantara  2022      Rishab Shetty  Rishab Shetty, Sapthami Gowda, Kishore Kumar G...     9.3            148     UA  Gross Unkown     Action   Adventure,  Drama
1                                The Dark Knight  2008  Christopher Nolan  Christian Bale, Heath Ledger, Aaron Eckhart, M...     9.0            152     UA      $534.86M     Action       Crime,  Drama
2  The Lord of the Rings: The Return of the King  2003      Peter Jackson  Elijah Wood, Viggo Mortensen, Ian McKellen, Or...     9.0            201      U      $377.85M     Action   Adventure,  Drama


In [14]:
# ---  ПРЕДОБРАБОТКА ДАННЫХ (Обновленный) ---

#  ПРОВЕРКА: Есть ли вообще пропуски?
print("📊 Статистика пропусков ДО обработки:")
cols_to_check = ['main_genre', 'side_genre', 'Rating']
missing_counts = df[cols_to_check].isnull().sum()
print(missing_counts)

# Выводим сообщение, чтобы ты понимал, нужно ли было исправление
if missing_counts.sum() == 0:
    print("\n✨ Отлично! Пропусков нет, но мы оставим код заполнения на случай сбоев в будущем.")
else:
    print(f"\n⚠️ Найдено {missing_counts.sum()} пропусков. Сейчас мы их заполним.")

print("-" * 40)

# 1. Работа с пропусками (Missing Values)
# Заполняем пустые жанры словом "Unknown", чтобы не было ошибок
df['main_genre'] = df['main_genre'].fillna('Unknown')
df['side_genre'] = df['side_genre'].fillna('Unknown')

# Заполняем пустые рейтинги средним значением
avg_rating = df['Rating'].mean()
df['Rating'] = df['Rating'].fillna(avg_rating)
print(f"✅ Обработка завершена. (Для справки: средний рейтинг = {avg_rating:.2f})")

# 2. Кодирование жанров (One-Hot Encoding)
# Превращаем текстовые колонки жанров в таблицу из 0 и 1
genre_features = df[['main_genre', 'side_genre']]
X_genres = pd.get_dummies(genre_features, prefix=['main', 'side'])
print(f"✅ Жанры закодированы. Теперь у нас {X_genres.shape[1]} колонок с жанрами.")

# 3. Масштабирование рейтинга (StandardScaler)
scaler = StandardScaler()
# reshape(-1, 1) нужен, так как мы берем одну колонку
rating_scaled = scaler.fit_transform(df[['Rating']])

# Создаем DataFrame для рейтинга
X_rating = pd.DataFrame(rating_scaled, columns=['Rating_Scaled'], index=df.index)
print("✅ Рейтинг масштабирован.")

# 4. Объединение всего в одну таблицу
X_final = pd.concat([X_genres, X_rating], axis=1)

print("\n🎉 Данные готовы для обучения!")
print(f"Итоговая таблица для модели имеет размер: {X_final.shape}")
print("\nПервые 3 строки подготовленных данных:")
print(X_final.head(3))

📊 Статистика пропусков ДО обработки:
main_genre    0
side_genre    0
Rating        0
dtype: int64

✨ Отлично! Пропусков нет, но мы оставим код заполнения на случай сбоев в будущем.
----------------------------------------
✅ Обработка завершена. (Для справки: средний рейтинг = 6.76)
✅ Жанры закодированы. Теперь у нас 157 колонок с жанрами.
✅ Рейтинг масштабирован.

🎉 Данные готовы для обучения!
Итоговая таблица для модели имеет размер: (5562, 158)

Первые 3 строки подготовленных данных:
   main_Action  main_Adventure  main_Animation  main_Biography  main_Comedy  main_Crime  main_Drama  main_Fantasy  main_Film-Noir  main_Horror  main_Musical  main_Mystery  main_Western  side_ Action,  Adventure  side_ Action,  Comedy  side_ Action,  Crime  side_ Action,  Drama  side_ Action,  Horror  side_ Adventure  side_ Adventure,  Biography  side_ Adventure,  Comedy  side_ Adventure,  Crime  side_ Adventure,  Drama  side_ Adventure,  Family  side_ Adventure,  Fantasy  side_ Adventure,  History  side_

In [15]:
# ---  ОБУЧЕНИЕ МОДЕЛЕЙ  ---

# Мы создадим словарь (dict), чтобы хранить там все наши модели
# Это позволит легко обращаться к ним по имени
models = {}

print("Начинаем обучение моделей...")

# ---------------------------------------------------------
# МОДЕЛЬ 1: KNN с Косинусным расстоянием (Cosine)
# ---------------------------------------------------------
# Эта модель отлично подходит для текстовых данных и жанров.
# Она смотрит на "направление" (насколько похож состав жанров).
print("1. Обучаем модель с метрикой Cosine...")
model_cosine = NearestNeighbors(n_neighbors=6, metric='cosine')
model_cosine.fit(X_final) # "Скармливаем" модели наши подготовленные данные
models['Cosine'] = model_cosine

# ---------------------------------------------------------
# МОДЕЛЬ 2: KNN с Евклидовым расстоянием (Euclidean)
# ---------------------------------------------------------
# Классическая геометрия (расстояние по прямой).
# Она будет более чувствительна к разнице в Рейтинге.
print("2. Обучаем модель с метрикой Euclidean...")
model_euclidean = NearestNeighbors(n_neighbors=6, metric='euclidean')
model_euclidean.fit(X_final)
models['Euclidean'] = model_euclidean

# ---------------------------------------------------------
# МОДЕЛЬ 3: KNN с Манхэттенским расстоянием (Manhattan)
# ---------------------------------------------------------
# Расстояние "городских кварталов" (сумма разниц по модулю).
# Часто работает лучше Евклидовой, когда очень много колонок (как у нас с жанрами).
print("3. Обучаем модель с метрикой Manhattan...")
model_manhattan = NearestNeighbors(n_neighbors=6, metric='manhattan')
model_manhattan.fit(X_final)
models['Manhattan'] = model_manhattan

print("\n🎉 Все три модели успешно обучены!")
print("Теперь мы можем сравнивать их рекомендации.")

Начинаем обучение моделей...
1. Обучаем модель с метрикой Cosine...
2. Обучаем модель с метрикой Euclidean...
3. Обучаем модель с метрикой Manhattan...

🎉 Все три модели успешно обучены!
Теперь мы можем сравнивать их рекомендации.


In [16]:
# ---  КОНТРОЛЬНЫЙ ТЕСТ (Проверка по индексу) ---

def recommend_movie_by_index(movie_index, n=5):
    """
    Тестовая функция: берет фильм по номеру строки (index)
    и показывает, что рекомендуют разные модели.
    """
    # Проверяем, существует ли такой индекс в таблице
    if movie_index >= len(df) or movie_index < 0:
        print(f"❌ Ошибка: Индекса {movie_index} нет в базе. Введите число от 0 до {len(df)-1}.")
        return

    # Достаем информацию об исходном фильме
    original_title = df.iloc[movie_index]['Movie_Title']
    print(f"\n🧪 ТЕСТ: Исходный фильм (индекс {movie_index}): {original_title}")
    print(f"   Жанры: {df.iloc[movie_index]['main_genre']}, {df.iloc[movie_index]['side_genre']}")
    print(f"   Рейтинг: {df.iloc[movie_index]['Rating']}")
    print("=" * 60)

    # Перебираем все наши модели (Cosine, Euclidean, Manhattan), которые мы обучили в Блоке 3
    for model_name, model in models.items():
        print(f"\n🧠 Рекомендации от модели: {model_name}")
        print("-" * 30)

        # Ищем соседей (используем X_final, который мы подготовили в Блоке 2)
        # model.kneighbors возвращает расстояния и индексы соседей
        distances, indices = model.kneighbors([X_final.iloc[movie_index]])

        # Выводим результаты (пропускаем 0-й элемент, т.к. это сам фильм)
        for i in range(1, len(indices[0])):
            idx = indices[0][i]

            # Достаем данные из оригинального датафрейма df по найденному индексу
            title = df.iloc[idx]['Movie_Title']
            main_g = df.iloc[idx]['main_genre']
            side_g = df.iloc[idx]['side_genre']
            rating = df.iloc[idx]['Rating']
            dist = distances[0][i]

            # Выводим информацию
            print(f"{i}. {title}")
            print(f"   Жанры: {main_g}, {side_g} | ★ Рейтинг: {rating} | Дистанция: {dist:.4f}")

# --- ЗАПУСК ТЕСТА ---
movie_to_test = 50 # Можешь поменять это число на любое другое (например, 0, 100, 250)
print(f"Запускаем тестовую рекомендацию для фильма с индексом {movie_to_test}...")

recommend_movie_by_index(movie_to_test)

Запускаем тестовую рекомендацию для фильма с индексом 50...

🧪 ТЕСТ: Исходный фильм (индекс 50): Indiana Jones and the Last Crusade
   Жанры: Action,  Adventure
   Рейтинг: 8.2

🧠 Рекомендации от модели: Cosine
------------------------------
1. Raiders of the Lost Ark
   Жанры: Action,  Adventure | ★ Рейтинг: 8.4 | Дистанция: 0.0021
2. Batman
   Жанры: Action,  Adventure | ★ Рейтинг: 7.5 | Дистанция: 0.0497
3. Indiana Jones and the Temple of Doom
   Жанры: Action,  Adventure | ★ Рейтинг: 7.5 | Дистанция: 0.0497
4. Kantara
   Жанры: Action,  Adventure,  Drama | ★ Рейтинг: 9.3 | Дистанция: 0.1904
5. The Lord of the Rings: The Return of the King
   Жанры: Action,  Adventure,  Drama | ★ Рейтинг: 9.0 | Дистанция: 0.1937

🧠 Рекомендации от модели: Euclidean
------------------------------
1. Raiders of the Lost Ark
   Жанры: Action,  Adventure | ★ Рейтинг: 8.4 | Дистанция: 0.2134
2. Batman
   Жанры: Action,  Adventure | ★ Рейтинг: 7.5 | Дистанция: 0.7470
3. Indiana Jones and the Temple of Doo

In [17]:
# ---  ИНТЕРАКТИВНЫЙ ПОИСК ПО НАЗВАНИЮ ---

def get_movie_recommendations_by_title(partial_title):
    """
    Ищет фильм по частичному названию и выдает рекомендации.
    """
    # 1. Поиск совпадений в датасете (регистронезависимый)
    # Ищем, содержит ли колонка Movie_Title введенный текст
    matches = df[df['Movie_Title'].str.contains(partial_title, case=False, na=False)]

    if len(matches) == 0:
        print(f"❌ Фильм с названием, содержащим '{partial_title}', не найден.")
        return

    # 2. Если нашли несколько, берем первый (самый вероятный)
    if len(matches) > 1:
        print(f"🔎 Найдено несколько фильмов ({len(matches)}). Используем первый из списка:")
        print(f"   -> {matches.iloc[0]['Movie_Title']} (Жанры: {matches.iloc[0]['main_genre']}, {matches.iloc[0]['side_genre']})")
    else:
        print(f"✅ Фильм найден: {matches.iloc[0]['Movie_Title']}")

    # 3. Получаем индекс найденного фильма
    movie_index = matches.index[0]

    # 4. Вызываем ту же функцию рекомендации, которую мы использовали в Блоке 5
    # Она покажет результаты по всем трем моделям
    recommend_movie_by_index(movie_index)

# --- ЗАПУСК ВВОДА ---
print("\n👇 Введите название фильма на английском (например: Batman, Matrix, Shrek)")
user_input = input("Название фильма: ") # Поле для ввода появится после запуска

if user_input.strip():
    get_movie_recommendations_by_title(user_input)
else:
    print("⚠️ Вы ничего не ввели.")


👇 Введите название фильма на английском (например: Batman, Matrix, Shrek)
Название фильма: Matrix
🔎 Найдено несколько фильмов (4). Используем первый из списка:
   -> The Matrix (Жанры: Action,  Sci-Fi)

🧪 ТЕСТ: Исходный фильм (индекс 6): The Matrix
   Жанры: Action,  Sci-Fi
   Рейтинг: 8.7

🧠 Рекомендации от модели: Cosine
------------------------------
1. Terminator 2: Judgment Day
   Жанры: Action,  Sci-Fi | ★ Рейтинг: 8.6 | Дистанция: 0.0003
2. The Terminator
   Жанры: Action,  Sci-Fi | ★ Рейтинг: 8.1 | Дистанция: 0.0162
3. Kantara
   Жанры: Action,  Adventure,  Drama | ★ Рейтинг: 9.3 | Дистанция: 0.1371
4. The Lord of the Rings: The Return of the King
   Жанры: Action,  Adventure,  Drama | ★ Рейтинг: 9.0 | Дистанция: 0.1453
5. The Dark Knight
   Жанры: Action,  Crime,  Drama | ★ Рейтинг: 9.0 | Дистанция: 0.1453

🧠 Рекомендации от модели: Euclidean
------------------------------
1. Terminator 2: Judgment Day
   Жанры: Action,  Sci-Fi | ★ Рейтинг: 8.6 | Дистанция: 0.1067
2. The Term